# Лекция №3 (часть 2)

## Распиливание больших функций

### Трагический пример

In [1]:
def long_function():
    x = "very important info"

    ...

    for x in range(10):
        ...
    
    ...

    return x

In [2]:
# 😱 😱 😱
long_function() == "very important info"

False

В предыдущем примере мы нечаянно перезаписали важную информацию.

Практический вывод: __количество переменных в функции должно быть небольшим и обозримым__.

Для этого связанные куски кода должны выноситься в отдельные функции.

In [3]:
def auxiliary_function():
    ...
    for x in range(10):
        ...
    ...

def main_function():
    x = "very important info"
    ...
    auxiliary_function()
    ...
    return x

In [4]:
main_function() == "very important info"

True

### Стратегии выделения вспомогательных функций

Рассмотрим пример:

In [5]:
def another_long_function():
    "Пример функции с несколькими вложенными циклами."
    with open("output.txt", "w", encoding="utf-8") as file:
        for x in range(100):
            for y in range(200):
                z = x + y
                message = f"{x} + {y} = {z}"
                file.write(message)

Мы можем вынести самые внутренние части цикла во __вспомогательную функцию__.

In [6]:
def write_sum(file, x, y):
    "Вспомогательная функция со внутренней частью цикла."
    z = x + y
    message = f"{x} + {y} = {z}"
    file.write(message)

def main_function_1():
    "Главная функция со внешней частью цикла."
    with open("output.txt", "w", encoding="utf-8") as file:
        for x in range(100):
            for y in range(200):
                write_sum(file, x, y)  # отсюда мы вынули кусок кода

Или мы можем выносить самые внешние части цикла во __вспомогательный итератор__:

In [7]:
def create_iterator():
    """
    Вспомогательная функция, создающая итератор.
    (Итератор выполняет внешнюю часть цикла.)
    """
    with open("output.txt", "w", encoding="utf-8") as file:
        for x in range(100):
            for y in range(200):
                yield file, x, y

def main_function_2():
    "Главная функция со внутренней частью цикла."
    it = create_iterator()
    for (file, x, y) in it:  # отсюда мы вынули кусок кода
        z = x + y
        message = f"{x} + {y} = {z}"
        file.write(message)

## Итераторы

Если внутри функции есть хотя бы одно ключевое слово `yield`, то она выполняется _лениво_:
* при вызове функции создаётся и возвращается итератор,
* итератор помнит своё внутреннее пространство имён,
* итератор помнит то место в коде, где приостановилось выполнение функции,
* при переборе элементов итератора в цикле `for`:
  * функция выполняется от последней точки остановки до ближайшего слова `yield`,
  * если процесс выполнения дошёл до конца функции или до `return`, итератор исчерпан.

In [8]:
def create_iterator():

    x = 1
    print( f"Inside iterator: {locals()}" )
    yield x

    y = 2
    print( f"Inside iterator: {locals()}" )
    yield y

    z = 3
    print( f"Inside iterator: {locals()}" )
    yield z

    return "Iteration stops here."  # 🙅 🙅 🙅

    w = 5
    print( "This code never runs.")
    yield w

In [9]:
it = create_iterator()
print(it)

<generator object create_iterator at 0x111c81990>


In [10]:
next(it)

Inside iterator: {'x': 1}


1

In [11]:
next(it)

Inside iterator: {'x': 1, 'y': 2}


2

In [12]:
next(it)

Inside iterator: {'x': 1, 'y': 2, 'z': 3}


3

Внутри итератора была строка `return "Iteration stops here."`

Ключевое слово `return` внутри итератора может положить сообщение внутрь `StopIteration`.

In [14]:
next(it)

StopIteration: 

## Исключения

### Неудобный пример

In [15]:
def get_number_from_user() -> int | None:
    """
    Запросить число у пользователя.
    Если пользователь ввёл не число, вернуть None.
    """
    user_input = input("Enter a number: ")
    if user_input.isdigit():
        return int(user_input)
    else:
        return None


def add_10(num: int | None) -> int | None:
    """
    Прибавить к числу 10 (если это число).
    """
    if num is None:
        return None
    else:
        return num + 10


def twice(num: int | None) -> int | None:
    """
    Умножить число на 2 (если это число).
    """
    if num is None:
        return None
    else:
        return num * 2

def main():
    "Сложное математическое выражение."
    num1 = twice(add_10(get_number_from_user()))
    num2 = add_10(twice(get_number_from_user()))
    if num1 is None or num2 is None:
        return 42
    else:
        return num1 + num2

Бóльшая часть кода — это механические операции по проверке полученного значения:
```python
    if num is None:
            return None
    else:
        ...
```

Чем это плохо?
* Занимает много места, отвлекает внимание от важных вещей.
* Легко ошибиться и забыть такую проверку.
* Просто бесит.

### Удобный пример

In [16]:
def get_number_from_user() -> int:
    """
    Запросить число у пользователя.
    Если пользователь ввёл не число, кинуть исключение ValueError.
    """
    user_input = input("Enter a number: ")
    if user_input.isdigit():
        return int(user_input)
    else:
        raise ValueError  # 🚀


def add_10(num: int) -> int:
    "Прибавить к числу 10."
    return num + 10


def twice(num: int | None) -> int:
    "Умножить число на 2 (если это число)."
    return num * 2

def main():
    "Сложное математическое выражение."
    try:
        num1 = twice(add_10(get_number_from_user()))
        num2 = add_10(twice(get_number_from_user()))
        return num1 + num2
    except ValueError:  # 🧑🏼‍🚀
        return 42

Брошенное исключение пролетает через стэк вызовов, пока не долетит до точки перехвата (`except`).

В нашем примере:
```python
   try:
      num1 = twice(add_10(get_number_from_user()))
   except ValueError:
      ...
```

Исключение, вылетевшее из `get_number_from_user`, пролетает через `add_10` и `twice`, а затем перехватывается в `main`.

Функции `add_10` и `twice` __не должны ничего знать об этом__. Они пишутся просто и естественно.

### Бонусный пример

Как работает цикл `for`.

In [17]:
def automatic_iterator():
    "Автоматическая размотка итератора в цикле `for`."
    for item in range(10):
        print(item, end=" ")

In [18]:
def handmade_iterator():
    "Размотка итератора своими руками."
    it = iter(range(10))
    while True:
        try:
            item = next(it)
            print(item, end=" ")
        except StopIteration:
            break

In [19]:
automatic_iterator()

0 1 2 3 4 5 6 7 8 9 

In [20]:
handmade_iterator()

0 1 2 3 4 5 6 7 8 9 